In [6]:

import os
from pandas.core.frame import DataFrame
import tkinter
import tkinter.messagebox
from fuzzywuzzy import process
# Get the files' path
dirlast = os.path.abspath('..')
def readDir(dirPath):
    '''Get all file name under the dirpath
    
    dirpath come from the requirements.'''
    allFiles = []
     #Loop to get all file names.
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        for f in fileList:
            if(f!="extraction" and f!=".DS_Store"):    
             f = dirPath+'/'+f
             allFiles.append(f)
    return allFiles 

# fix search Extract
def extract(text):
  '''The fix search method to return the fix search result.
    
    search text comes from user input.'''
  # The path of the output files.
  txt = dirlast+ '/output'+'/'+text + '.txt'
  table =dirlast+ '/table'+'/'+ text + '.csv'
  output = open(txt, 'w')
  logk = []
  reaction = []
  datafrom = []
  #Read files 
  for file in readDir(dirlast+"/data"):
    f = open(file,'r')
    # Save the dataset this search result from 
    dataset = file[file.find("data")+5:5000].strip(".txt")
    check = 0
    lines = f.readlines()
    linenumber = 0
    # Begin the search
    for i in range(len(lines)-2):
      # First ensure the search text is in this line, and then ensure it is a reaction by "="
      if text in lines[i] and  "=" in lines[i]:
          # There is two format of logk, neighboor and two lines behind the last one.
          if "log_k"in lines[i+1]:
           linenumber = i+1
          if "log_k"in lines[i+2]:
           linenumber = i+2
          # Extract logK according to the different situation
          if "log_k"in lines[i+1] or "log_k"in lines[i+2]:
          # Get the reaction
           react = lines[i].strip().strip('#').strip()
           reaction.append(react)
           # Get logk
           k = lines[linenumber][lines[linenumber].find("k")+1:50].strip()
           # only save the logk when k not euaql to  0
           if k != 0:
            logk.append(k)
            datafrom.append(dataset)
            # Write the output file.
            if check == 0:
                output.write("\n")
                output.write("----------------------------------------------------")
                output.write("\n")
                output.write("Find from: "+dataset )
                output.write("\n")
                output.write("----------------------------------------------------")
                output.write("\n")
                check = check+1
            output.write("\n")
            output.write("\n")
            output.write(lines[i].strip().strip('#').strip())
            output.write("\n")
            output.write(lines[linenumber].strip().strip('#').strip())
            # Save the data into dataframe 
            data = {"Reaction":reaction,"logk":logk,"database":datafrom}
            dataframe=DataFrame(data)
            # Save the dataframe to the csv file.
            dataframe.to_csv(table,index=False,sep=',')
  output.close()
  fileList = os.listdir(dirlast+ '/output')
  for fl in fileList:
     if os.path.getsize(dirlast+ '/output/'+fl) == 0:
                os.remove(dirlast+ '/output/'+fl)
                return False
  return True


def fuzzyextract(text,number = 100):
  '''The fuuzy search method to return the fuzzy search result.
    
    search text and numbers comes from user input.'''
  # The path of the output files.
  table =dirlast+ '/table'+'/'+ text + '.csv'
  # List conclude all logk value.
  logk = []
  # List conclude all reaction text.
  reaction = []
  # List conclude all dataset the reaction from.
  datafrom = []
  logknew = []
  reactionnew = []
  datafromnew = []
  name = []
  namenew = []
  
  for file in readDir(dirlast+"/data"):
    f = open(file,'r',encoding='utf-8')
    dataset = file[file.find("data")+5:50000].strip(".txt")
    check = 0
    ks = 0.02
    lines = f.readlines()
    checkifexist = False
    linenumber = 0
    for i in range(len(lines)-2):
      checks = False
      # Save all the reactions
      if  "=" in lines[i]:
         # if "E" in lines[i]:
          if "log_k"in lines[i+1]:
           linenumber = i+1
           checks = True
          if "log_k"in lines[i+2]:
           linenumber = i+2
           checks = True
          ks = lines[linenumber][lines[linenumber].find("k")+1:50].strip()
          
          if "#" in ks:
               ks = ks[0: ks.find("#")-2]
          if checks==True:
           
           react = lines[i].strip().strip('#').strip()
           named = lines[i-1].strip()
           reaction.append(react)
           name.append(named)
           k = ks
           logk.append(k)
           datafrom.append(dataset)
  # Do fuzzy serach to the all reactions to match the search text.
  m = process.extract(text, reaction, limit=number)  
  # Save the serach result.
  for j in range(len(reaction)):
      for best in m:
          if best[0] in reaction[j]:
            if check== 0:
              reactionnew.append(best[0])
              datafromnew.append(datafrom[j])
              logknew.append(logk[j])
              namenew.append(name[j])
              check = check+1
            else:
                for o in range(len(reactionnew)):
                    if best[0] in reactionnew[o] and logk[j] in logknew[o]:
                        checkifexist = True
                if checkifexist == False:
                       reactionnew.append(best[0])
                       datafromnew.append(datafrom[j])
                       logknew.append(logk[j])
                       namenew.append(name[j]) 
                checkifexist = False
               
  data = {"Reaction":reactionnew,"logk":logknew,"database":datafromnew,"name":namenew}
           
  dataframe=DataFrame(data)
        
  dataframe.to_csv(table,index=False,sep=',')      
  fileList = os.listdir(dirlast+ '/table')
  for fl in fileList:
     if os.path.getsize(dirlast+ '/table/'+fl) == 0:
                os.remove(dirlast+ '/table/'+fl)
                return False
  return True

In [4]:
pip install fuzzywuzzy

In [7]:

import tkinter
import tkinter.messagebox
from extraction import fuzzyextract, extract

def searchui():
    def fuzzy():
        searchtext  = entrytext.get()
        num = int(entrynum.get())
        if fuzzyextract(searchtext ,num):
            tkinter.messagebox.showinfo(title='userlogin', message='ok')
        else:
            tkinter.messagebox.showerror(title='userlogin', message='Error')
    
    #def normal():
     #  searchtext = entrytext2.get()
      #  if extract(searchtext):
       #     tkinter.messagebox.showinfo(title='userlogin', message='ok')
      #  else:
      #      tkinter.messagebox.showerror(title='userlogin', message='Error')
        
    def cancel():
        vartext1 .set('')
        varnum.set('')
        vartext2.set('')
     
    window = tkinter.Tk()
    window.title('SEARCH')
    window.geometry('750x300')
    vartext1 = tkinter.StringVar(value='')
    varnum = tkinter.StringVar(value='')
    vartext2 = tkinter.StringVar(value='')
    
    # Create the label for fuzzy search
    labelName = tkinter.Label(window, text='Search text', justify=tkinter.RIGHT, width=80)
    
    labelName.place(x=10, y=5, width=80, height=20)
    # Create the text box and the input text.
    entrytext = tkinter.Entry(window, width=80, textvariable=vartext1)
    entrytext.place(x=100, y=5, width=80, height=20)
    labelPwd = tkinter.Label(window, text='Number:', justify=tkinter.RIGHT, width=80)
    labelPwd.place(x=10, y=30, width=80, height=20)
    # Create the text box for the fuzzy number. 
    entrynum = tkinter.Entry(window,  width=80, textvariable=varnum)
    entrynum.place (x=100, y=30, width=80, height=20)
    
    # Create the buttons and the link related to them
    buttonOk = tkinter.Button(window, text='Search', command=fuzzy)
    buttonOk.place(x=10, y=70, width=100, height=20)
    buttonCancel = tkinter.Button(window, text='Cancel', command=cancel)
    buttonCancel.place(x=120, y=70, width=100, height=20)
    
    # Create the label for exact search 
    #labelName = tkinter.Label(window, text='Search text', justify=tkinter.RIGHT, width=80)
    #labelName.place(x=10, y=200, width=80, height=20)
    #entrytext2 = tkinter.Entry(window, width=80, textvariable=vartext2)
    #entrytext2.place(x=100, y=200, width=80, height=20)
    #buttonOk = tkinter.Button(window, text='Search', command=normal)
    #buttonOk.place(x=10, y=270, width=50, height=20)
    #buttonCancel = tkinter.Button(window, text='Cancel', command=cancel)
    #buttonCancel.place(x=120, y=270, width=50, height=20)
    # Loop show the interface
    labelName = tkinter.Label(window, text='Input related text, for example zncit for Zncitrate', justify=tkinter.RIGHT, width=80)
    
    labelName.place(x=180, y=5, width=300, height=20)
    
    labelName = tkinter.Label(window, text='Input the related result numbers you want to return (suggested range : 15-25)', justify=tkinter.RIGHT, width=80)
    
    labelName.place(x=170, y=30, width=500, height=20)
    
    labelName = tkinter.Label(window, text='Press search to complete the search or cancel to empty the input boxes.', justify=tkinter.RIGHT, width=80)
    
    labelName.place(x=50, y=100, width=480, height=20)
    
    #labelName = tkinter.Label(window, text='Input the specific text you want to search e.g ZnCitrate, Make sure the low and upper case is exactly the same.', justify=tkinter.RIGHT, width=80)
    
    #labelName.place(x=10, y=220, width=600, height=20)
    
    window .mainloop()

searchui()